<a href="https://colab.research.google.com/github/windopper/NerualNetworkPracticeInJupyter/blob/main/RecurrentNeuralNetwork/charRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문자 단위 RNN(Char RNN)

입력과 출력의 단위가 단어 벡터였던 것을 문자 벡터로 변경하여 구현하는 RNN

다 대 다 (Many-to-Many) 구조로 구현한 경우, 다 대 일(Many-to-One) 구조로 구현한 경우 두 가지를 살펴보겠음

먼저, 다 대 다 구조

# 문자 단위 RNN 언어 모델

데이터 로드, 특수 문자 제거, 단어 소문자화 전처리 시행

In [55]:
import numpy as np
import urllib.request
from tensorflow.keras.utils import to_categorical

urllib.request.urlretrieve("http://www.gutenberg.org/files/11/11-0.txt", filename="11-0.txt")

f = open('11-0.txt', 'rb')
sentences = []
for sentence in f:
  sentence = sentence.strip() # strip() 을 통해 \r, \n 을 제거
  sentence = sentence.lower() # 소문자화
  sentence = sentence.decode('ascii', 'ignore') # \x32\x80\x99 등과 같은 바이트 열 제거

  if len(sentence) > 0:
    sentences.append(sentence)

f.close()

sentences[:5]

['the project gutenberg ebook of alices adventures in wonderland, by lewis carroll',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. you may copy it, give it away or re-use it under the terms',
 'of the project gutenberg license included with this ebook or online at']

원소가 문자열로 구성되어있지만 토큰화는 된 상태가 아니기에 하나의 문자열로 통일

In [56]:
total_data = ' '.join(sentences)
print('문자열의 길이 또는 총 문자의 개수 : %d' % len(total_data))

문자열의 길이 또는 총 문자의 개수 : 159484


In [57]:
print(total_data[:200])

the project gutenberg ebook of alices adventures in wonderland, by lewis carroll this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with


위 문자열을 문자 집합으로 바꾸면

In [65]:
char_vocab = sorted(list(set(total_data)))
print('문자 집합의 크기 :', len(char_vocab))
vocab_size = len(char_vocab)

문자 집합의 크기 : 56


문자에 고유한 정수 부여

In [59]:
char_to_index = dict((char, index) for index, char in enumerate(char_vocab))
print('문자 집합 : ', char_to_index)

문자 집합 :  {' ': 0, '!': 1, '"': 2, '#': 3, '$': 4, '%': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, '_': 29, 'a': 30, 'b': 31, 'c': 32, 'd': 33, 'e': 34, 'f': 35, 'g': 36, 'h': 37, 'i': 38, 'j': 39, 'k': 40, 'l': 41, 'm': 42, 'n': 43, 'o': 44, 'p': 45, 'q': 46, 'r': 47, 's': 48, 't': 49, 'u': 50, 'v': 51, 'w': 52, 'x': 53, 'y': 54, 'z': 55}


정수로 부터 문자를 리턴하는 딕셔너리 제작

In [60]:
index_to_char = {}
for char, index in char_to_index.items():
  index_to_char[index] = char

샘플 제작

In [61]:
seq_length = 60

n_samples = int(np.floor((len(total_data)-1)/ seq_length))
print('샘플의 수 : {}'.format(n_samples))

샘플의 수 : 2658


In [62]:
train_x = []
train_y = []

for i in range(n_samples):
  # 0:60 -> 60:120 -> 120:180로 loop를 돌면서 문장 샘플을 1개씩 pick.
  x_sample = total_data[i * seq_length: (i+1) * seq_length]

  # 정수 인코딩
  x_encoded = [char_to_index[c] for c in x_sample]
  train_x.append(x_encoded)

  # 오른쪽으로 한칸 쉬프트
  y_sample = total_data[i * seq_length +1 : (i+1) * seq_length+1]
  y_encoded = [char_to_index[c] for c in y_sample]
  train_y.append(y_encoded)

train_x 와 train_y 의 첫번째 샘플을 출력하면

In [63]:
print('x 데이터의 첫번째 샘플 :', train_x[0])
print('y 데이터의 첫번째 샘플 :', train_y[0])

print('-'*50)
print('x 데이터의 첫번재 샘플 디코딩 :', [index_to_char[i] for i in train_x[0]])
print('y 데이터의 첫번째 샘플 디코딩 :', [index_to_char[i] for i in train_y[0]])

x 데이터의 첫번째 샘플 : [49, 37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30]
y 데이터의 첫번째 샘플 : [37, 34, 0, 45, 47, 44, 39, 34, 32, 49, 0, 36, 50, 49, 34, 43, 31, 34, 47, 36, 0, 34, 31, 44, 44, 40, 0, 44, 35, 0, 30, 41, 38, 32, 34, 48, 0, 30, 33, 51, 34, 43, 49, 50, 47, 34, 48, 0, 38, 43, 0, 52, 44, 43, 33, 34, 47, 41, 30, 43]
--------------------------------------------------
x 데이터의 첫번재 샘플 디코딩 : ['t', 'h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'o', 'f', ' ', 'a', 'l', 'i', 'c', 'e', 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ', 'i', 'n', ' ', 'w', 'o', 'n', 'd', 'e', 'r', 'l', 'a']
y 데이터의 첫번째 샘플 디코딩 : ['h', 'e', ' ', 'p', 'r', 'o', 'j', 'e', 'c', 't', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e',

train_x와 train_y에 대하여 원-핫 인코딩 진행

In [64]:
train_x = to_categorical(train_x)
train_y = to_categorical(train_y)

print('train_x의 크기', train_x.shape)
print('train_y의 크기', train_y.shape)

train_x의 크기 (2658, 60, 56)
train_y의 크기 (2658, 60, 56)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

hidden_units = 256

model = Sequential()
model.add(LSTM(hidden_units, input_shape=(None, train_x.shape[2]), return_sequences=True))
model.add(LSTM(hidden_units, return_sequences=True))
model.add(TimeDistributed(Dense(len(char_vocab), activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=40, verbose=2)


In [ ]:
def sentence_generation(model, length):
    # 문자에 대한 랜덤한 정수 생성
    ix = [np.random.randint(vocab_size)]

    # 랜덤한 정수로부터 맵핑되는 문자 생성
    y_char = [index_to_char[ix[-1]]]
    print(ix[-1],'번 문자',y_char[-1],'로 예측을 시작!')

    # (1, length, 55) 크기의 X 생성. 즉, LSTM의 입력 시퀀스 생성
    X = np.zeros((1, length, vocab_size))

    for i in range(length):
        # X[0][i][예측한 문자의 인덱스] = 1, 즉, 예측 문자를 다음 입력 시퀀스에 추가
        X[0][i][ix[-1]] = 1
        print(index_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
        y_char.append(index_to_char[ix[-1]])
    return ('').join(y_char)

result = sentence_generation(model, 100)
print(result)